In [ ]:
import os
import sys
import json
import numpy as np
import time
import cv2
from PIL import Image, ImageDraw
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapOnImage

In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")


# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class NageursConfig(Config):
    """Configuration pour un entrainement fait sur un nageur.
    Model ispiré de l'original mais adapté à mon GPU
    """
    # Give the configuration a recognizable name
    NAME = "nageur"

    # Batch size = GPU*Img (mieux si élevé mais dépend de la mémoire disponible)
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Nombres de Classes
    NUM_CLASSES = 1 + 1  # background + 1 (nageur)
    #ATTENTION : Le background est généré automatiquement, si dans le fichier JSON des annotations il existe une classe background,
    #pensez à la supprimer. Sinon vous aurez une erreur.

    # Shap
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    #Depend de la mémoire, si peu de mémoire, bcps de step.
    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 5
    
    # choix du backbone, peut être modifié si on ajoute un autre backbone dans le fichier model.py
    BACKBONE = 'resnet50'
    #paramètre du RPN
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = NageursConfig()
config.display()

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            #if class_id < 1:
            #    print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
            #    return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            print (image_id)
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    print (image_file_name)
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

In [ ]:
#En entrée : path vers le fichier d'annotation, path vers le dossier contenant les images.
'''dataset_train = CocoLikeDataset()
dataset_train.load_data(ROOT_DIR + '/datasets/nageur/train/coco_annotations.json', ROOT_DIR + '/datasets/nageur/train/images')
dataset_train.prepare()
'''
dataset_train = CocoLikeDataset()
dataset_train.load_data(ROOT_DIR + '/datasets/nageur/val/coco_annotations.json', ROOT_DIR + '/datasets/nageur/val/images')
dataset_train.prepare()

dataset_val = CocoLikeDataset()
dataset_val.load_data(ROOT_DIR + '/datasets/nageur/val/coco_annotations.json', ROOT_DIR + '/datasets/nageur/val/images')
dataset_val.prepare()

In [ ]:
dataset = dataset_train
for i in range (5):
    image = dataset.load_image(i)
    mask, class_ids = dataset.load_mask(i)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
#Choix du début de l'entrainement pour le fine tuning
init_with = "coco"  #  coco or last

if init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [ ]:
# Train head Layers
# Les autres branches sont figées pendant l'entrainement

start_train = time.time()
model.train(dataset_train,
            dataset_val,            
            learning_rate=0.0001, 
            epochs=15, 
            layers='heads',
            augmentation = iaa.Sometimes(2/3,iaa.OneOf([iaa.Flipud(1)])))#augmentation miroire
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')

In [ ]:
# Fine tuning sur tout le réseau (avec un learning rate plus faible, on est la pour affiner seulement)

start_train = time.time()
model.train(dataset_train,
            dataset_val, 
            learning_rate=0.0001,
            epochs=30, 
            layers="all",
            augmentation = iaa.Sometimes(1/2,iaa.OneOf([iaa.Flipud(1)])))#augmentation miroire
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')